In [1]:
import torch
import torchvision
import math
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm_notebook
from torch.utils.tensorboard import SummaryWriter


from GWD_data import WheatDataset
from metric import calculate_image_precision
from utils import collate_fn, plot_boxes, get_model_name, format_prediction_string, remove_blanks, is_contain_blanks
from config import config


In [2]:
conf = config()
conf.DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
conf.BATH_SIZE = 4
torch.random.manual_seed(5)
writer = SummaryWriter("runs/Validate_RC") #


WD_Train = WheatDataset(conf, augmentation=False)
WD_Valid = WheatDataset(conf, augmentation=False, train=False)
WD_Train_Loader = torch.utils.data.DataLoader(WD_Train, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)
WD_Valid_Loader = torch.utils.data.DataLoader(WD_Valid, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)

In [3]:
import os
os.listdir('model/model_rc')

['fasterrcnn_resnet50_fpn_best.pth',
 'GWD.pt',
 'GWD_EPOCH_0_SCORE_0.0000_LOSS_237.9718.pt',
 'GWD_EPOCH_10_SCORE_0.0000_LOSS_206.7397.pt',
 'GWD_EPOCH_11_SCORE_0.0000_LOSS_205.6545.pt',
 'GWD_EPOCH_12_SCORE_0.0000_LOSS_205.1640.pt',
 'GWD_EPOCH_13_SCORE_0.0000_LOSS_205.4779.pt',
 'GWD_EPOCH_14_SCORE_0.0000_LOSS_205.5879.pt',
 'GWD_EPOCH_15_SCORE_0.0000_LOSS_204.1447.pt',
 'GWD_EPOCH_16_SCORE_0.0000_LOSS_201.6743.pt',
 'GWD_EPOCH_17_SCORE_0.0000_LOSS_204.0144.pt',
 'GWD_EPOCH_18_SCORE_0.0000_LOSS_202.8660.pt',
 'GWD_EPOCH_19_SCORE_0.0000_LOSS_201.4089.pt',
 'GWD_EPOCH_1_SCORE_0.0000_LOSS_222.6304.pt',
 'GWD_EPOCH_20_SCORE_0.0000_LOSS_200.7300.pt',
 'GWD_EPOCH_2_SCORE_0.0000_LOSS_218.9721.pt',
 'GWD_EPOCH_3_SCORE_0.0000_LOSS_216.0619.pt',
 'GWD_EPOCH_4_SCORE_0.0000_LOSS_214.6626.pt',
 'GWD_EPOCH_5_SCORE_0.0000_LOSS_211.9636.pt',
 'GWD_EPOCH_6_SCORE_0.0000_LOSS_211.5326.pt',
 'GWD_EPOCH_7_SCORE_0.0000_LOSS_208.4762.pt',
 'GWD_EPOCH_8_SCORE_0.0000_LOSS_208.8839.pt',
 'GWD_EPOCH_9_SCORE_0

In [4]:
model_list = [

 'GWD_EPOCH_9_SCORE_0.0000_LOSS_209.3289.pt',
 'GWD_EPOCH_10_SCORE_0.0000_LOSS_206.7397.pt',
 'GWD_EPOCH_11_SCORE_0.0000_LOSS_205.6545.pt',
 'GWD_EPOCH_12_SCORE_0.0000_LOSS_205.1640.pt',
 'GWD_EPOCH_13_SCORE_0.0000_LOSS_205.4779.pt',
 'GWD_EPOCH_14_SCORE_0.0000_LOSS_205.5879.pt',
 'GWD_EPOCH_15_SCORE_0.0000_LOSS_204.1447.pt',
 'GWD_EPOCH_16_SCORE_0.0000_LOSS_201.6743.pt',
 'GWD_EPOCH_17_SCORE_0.0000_LOSS_204.0144.pt',
 'GWD_EPOCH_18_SCORE_0.0000_LOSS_202.8660.pt',
 'GWD_EPOCH_19_SCORE_0.0000_LOSS_201.4089.pt'
 ]

# Testing Data

In [5]:
torch.cuda.empty_cache()


In [6]:
for i in tqdm_notebook(range(len(model_list))):
    
    EPOCH = int(model_list[i].split('_')[2])
    LOSS = float(model_list[i].split('_')[-1][:-3])
    
    GWD_Model = torch.load('model/model_rc/'+model_list[i])
    GWD_Model.eval()
    GWD_Model.to(conf.DEVICE)
    precisions = []
    
    for images, targets in tqdm_notebook(WD_Valid_Loader):
        try:
            images = [torch.tensor(image, dtype = torch.float32).to(conf.DEVICE) for image in images]
            preds = GWD_Model(images)

            for pred, target in zip(preds, targets):
                ground_truth = target['boxes']
                pred_boxes = pred['boxes'].to(torch.device('cpu')).detach().numpy()
                precisions.append(calculate_image_precision(ground_truth, pred_boxes, thresholds=conf.PRECISION_THRESH))
        except:
            print('weird shape error')
    
    test_score = np.mean(np.array(precisions))
    writer.add_scalar('Epoch_Vars/Model_Loss', LOSS, EPOCH)
    writer.add_scalar('Epoch_Vars/Validation_Score', test_score, EPOCH)
    
    GWD_Model=None
    torch.cuda.empty_cache()
    gc.collect()

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


# tRAINING dTA

In [3]:
model_list = [

 'GWD_EPOCH_74_SCORE_0.0000_LOSS_188.3118.pt',
 'GWD_EPOCH_75_SCORE_0.0000_LOSS_186.5710.pt',
 'GWD_EPOCH_76_SCORE_0.0000_LOSS_187.5649.pt',
 'GWD_EPOCH_77_SCORE_0.0000_LOSS_188.5023.pt',
 'GWD_EPOCH_78_SCORE_0.0000_LOSS_187.6333.pt',
 'GWD_EPOCH_79_SCORE_0.0000_LOSS_188.6617.pt',
 'GWD_EPOCH_80_SCORE_0.0000_LOSS_186.8159.pt',
 'GWD_EPOCH_81_SCORE_0.0000_LOSS_186.5748.pt',
 'GWD_EPOCH_82_SCORE_0.0000_LOSS_187.3791.pt',
 'GWD_EPOCH_83_SCORE_0.0000_LOSS_187.8456.pt',
 'GWD_EPOCH_84_SCORE_0.0000_LOSS_187.2457.pt',
 'GWD_EPOCH_85_SCORE_0.0000_LOSS_186.4667.pt',
 'GWD_EPOCH_86_SCORE_0.0000_LOSS_187.2145.pt',
 'GWD_EPOCH_87_SCORE_0.0000_LOSS_187.1556.pt',
 'GWD_EPOCH_88_SCORE_0.0000_LOSS_186.3744.pt',
 'GWD_EPOCH_89_SCORE_0.0000_LOSS_184.3004.pt',
 'GWD_EPOCH_90_SCORE_0.0000_LOSS_185.7923.pt',
 'GWD_EPOCH_91_SCORE_0.0000_LOSS_185.0747.pt',
 'GWD_EPOCH_92_SCORE_0.0000_LOSS_184.9123.pt',
 'GWD_EPOCH_93_SCORE_0.0000_LOSS_185.4093.pt',
 'GWD_EPOCH_94_SCORE_0.0000_LOSS_186.2040.pt',
 'GWD_EPOCH_95_SCORE_0.0000_LOSS_185.7346.pt',
 'GWD_EPOCH_96_SCORE_0.0000_LOSS_185.8636.pt',
 'GWD_EPOCH_97_SCORE_0.0000_LOSS_184.2307.pt',
 'GWD_EPOCH_98_SCORE_0.0000_LOSS_183.7612.pt',
 'GWD_EPOCH_99_SCORE_0.0000_LOSS_186.0609.pt']

In [4]:
torch.cuda.empty_cache()
for i in tqdm_notebook(range(len(model_list))):
    
    EPOCH = int(model_list[i].split('_')[2])
    LOSS = float(model_list[i].split('_')[-1][:-3])
    
    GWD_Model = torch.load('model/model_aug/'+model_list[i])
    GWD_Model.eval()
    GWD_Model.to(conf.DEVICE)
    precisions = []
    
    for images, targets in tqdm_notebook(WD_Train_Loader):
        try:
            images = [torch.tensor(image, dtype = torch.float32).to(conf.DEVICE) for image in images]
            preds = GWD_Model(images)

            for pred, target in zip(preds, targets):
                ground_truth = target['boxes']
                pred_boxes = pred['boxes'].to(torch.device('cpu')).detach().numpy()
                precisions.append(calculate_image_precision(ground_truth, pred_boxes, thresholds=conf.PRECISION_THRESH))
        except:
            print('weird shape error')
    
    train_score = np.mean(np.array(precisions))
    #writer.add_scalar('Epoch_Vars/Model_Loss', LOSS, EPOCH)
    writer.add_scalar('Epoch_Vars/Training_Score', train_score, EPOCH)
    
    GWD_Model=None
    torch.cuda.empty_cache()
    gc.collect()

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


In [39]:
i = [173, 3687, 4412, 113947, 117344, 118211, 121633, 121634, 147504, 147552]
f = pd.read_csv(conf.DATA_PATH+'/train.csv')
I = f[f.image_id == '893938464']
B = I[I.bbox=='[116.0, 177.0, 445.0, 133.0]']
B

,image_id,width,height,bbox,source
147552,893938464,1024,1024,"[116.0, 177.0, 445.0, 133.0]",arvalis_2


In [38]:
f.shape

(147793, 5)

In [40]:
I = f.drop(f.index[i])
I.shape

(147783, 5)